## GitHub ドキュメント同期ツール (Google Drive連携)

GitHub上の技術ドキュメントやリポジトリを自動的に取得し、Google Driveへ保存するツールです。
保存されたデータを **NotebookLM** や **Claude Projects** のデータソース活用の効率化を目的に作成しました。

### 使い方
1.  **設定**: 下記のフォームに、取得したいGitHubのURLを入力します。
    *   ブラウザのアドレスバーのURLをそのまま貼り付け可能です。
    *   特定のフォルダ（例: `.../tree/main/docs`）のみを指定して軽量に取得することもできます。
2.  **実行**: セルの左上にある再生ボタン（▶）をクリックします。
3.  **認証**: 「Google ドライブに接続」というポップアップが表示されたら、アクセスを許可してください。
:

In [ ]:
# ==============================================================================
# GitHub ドキュメント同期ツール
# ==============================================================================
#
# GitHub上のドキュメントやデータを取得し、Googleドライブへ自動保存します。
# 社内ナレッジの蓄積や、AI（NotebookLM / Claude）への読み込み用データ作成に最適です。
#
# 【使い方】
# 1. 以下のフォームに取得したいGitHubのURLを入力してください。
# 2. 「再生ボタン（▶）」を押して実行します。
# 3. Googleドライブへのアクセス許可を求められたら「接続」を選択してください。
#
# ==============================================================================

#@title 設定フォーム { display-mode: "form" }

#@markdown ### 1. 取得元 (GitHub URL)
#@markdown ブラウザのURLをそのまま貼り付けてください (最大5件)
repo_url_1 = "https://github.com/vercel/next.js/tree/canary/docs/01-app/01-getting-started" #@param {type:"string"}
repo_url_2 = "" #@param {type:"string"}
repo_url_3 = "" #@param {type:"string"}
repo_url_4 = "" #@param {type:"string"}
repo_url_5 = "" #@param {type:"string"}

#@markdown ### 2. 保存先設定 (Google Drive)
#@markdown マイドライブ配下のフォルダ名を指定してください。<br>
#@markdown ※新規の場合は自動作成され、既存の場合はそのフォルダ内に保存されます。
drive_folder = "GitHub_Documents" #@param {type:"string"}

#@markdown ### 3. オプション
#@markdown 処理の詳細ログを表示する
verbose_mode = False #@param {type:"boolean"}

# ==============================================================================
# システム処理 (これより下は編集不要です)
# ==============================================================================

import os
import sys
import shutil
import re
from datetime import datetime
import time

def log(msg, indent=0):
    prefix = "  " * indent
    timestamp = datetime.now().strftime("%H:%M:%S")
    print(f"[{timestamp}] {prefix}{msg}")

def print_separator(char="-", length=70):
    print(char * length)

print("\n")
print_separator("=")
log("同期処理を開始します")
print_separator("=")

# ------------------------------------------------------------------------------
# 1. Google Drive 接続
# ------------------------------------------------------------------------------

try:
    from google.colab import drive
    log("Googleドライブに接続しています...")
    drive.mount('/content/drive', force_remount=False)
    drive_root = '/content/drive/MyDrive'
    log(f"接続完了: {drive_root}")
except Exception as e:
    log(f"【エラー】Googleドライブへの接続に失敗しました。\n{e}")
    sys.exit(1)

# ------------------------------------------------------------------------------
# 2. URL検証と解析
# ------------------------------------------------------------------------------

print_separator()
log("設定されたURLを確認しています...")

raw_urls = [url.strip() for url in [repo_url_1, repo_url_2, repo_url_3, repo_url_4, repo_url_5] if url.strip()]

if not raw_urls:
    log("【エラー】URLが入力されていません。設定フォームを確認してください。")
    sys.exit(1)

def parse_github_url(url):
    """Parse GitHub URL to extract repo, branch, and path."""
    patterns = [
        # /tree/branch/path
        (r'github\.com/([^/]+/[^/]+)/tree/([^/]+)/(.+)', lambda m: (m.group(1), m.group(2), m.group(3).rstrip('/'))),
        # /tree/branch
        (r'github\.com/([^/]+/[^/]+)/tree/([^/]+)/?$', lambda m: (m.group(1), m.group(2), '')),
        # root
        (r'github\.com/([^/]+/[^/]+)/?$', lambda m: (m.group(1), 'main', ''))
    ]

    for pattern, extractor in patterns:
        match = re.search(pattern, url)
        if match:
            repo, branch, path = extractor(match)
            return {'repo': repo, 'branch': branch, 'path': path, 'url': url}
    return None

targets = []
for url in raw_urls:
    parsed = parse_github_url(url)
    if parsed:
        targets.append(parsed)
        if verbose_mode:
            log(f"対象確認: {parsed['repo']} (branch: {parsed['branch']}, path: {parsed['path'] or '/'})", 1)
    else:
        log(f"【警告】無効な形式のURLが含まれています -> {url}")

if not targets:
    log("【エラー】有効な取得対象が見つかりませんでした。")
    sys.exit(1)

log(f"取得対象: {len(targets)} 件のリポジトリ")
print_separator()

# ------------------------------------------------------------------------------
# 3. データ取得・保存処理
# ------------------------------------------------------------------------------

def get_dir_stats(path):
    """Return file count, size, and extension summary."""
    count = 0
    size = 0
    exts = {}
    for root, _, files in os.walk(path):
        for f in files:
            count += 1
            fp = os.path.join(root, f)
            size += os.path.getsize(fp)
            ext = os.path.splitext(f)[1] or "拡張子なし"
            exts[ext] = exts.get(ext, 0) + 1
    return count, size, exts

def process_repository(target, index, total):
    repo = target['repo']
    branch = target['branch']
    path = target['path']
    repo_name = repo.split('/')[-1]

    log(f"[{index}/{total}] 処理中: {repo}")

    # Clone
    clone_base = f"/content/temp_{repo_name}"
    if os.path.exists(clone_base): shutil.rmtree(clone_base)

    log(f"GitHubからデータを取得しています...", 1)

    git_cmd_base = f"git clone --depth 1 --branch {branch} https://github.com/{repo} {clone_base}"

    # Sparse checkout if path is specified
    if path:
        cmd = f"{git_cmd_base} --filter=blob:none --sparse --quiet"
        ret = os.system(cmd)
        if ret == 0:
            os.chdir(clone_base)
            os.system(f"git sparse-checkout set {path}")
            os.chdir("/content")
            source_path = os.path.join(clone_base, path)
        else:
            source_path = None
    else:
        cmd = f"{git_cmd_base} --quiet"
        ret = os.system(cmd)
        source_path = clone_base if ret == 0 else None

    if not source_path or not os.path.exists(source_path):
        log("【失敗】データの取得に失敗しました。パスや権限を確認してください。", 1)
        return None

    # Stats
    f_count, f_size, f_exts = get_dir_stats(source_path)
    size_mb = f_size / (1024 * 1024)
    log(f"取得完了: {f_count} ファイル ({size_mb:.2f} MB)", 1)

    if verbose_mode and f_exts:
        top_exts = sorted(f_exts.items(), key=lambda x:x[1], reverse=True)[:3]
        log(f"内訳: {', '.join([f'{k}:{v}' for k,v in top_exts])}", 2)

    # Sync to Drive
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    dest_dir_name = f"{repo_name}_{timestamp}"
    dest_path = os.path.join(drive_root, drive_folder, dest_dir_name)

    log(f"Googleドライブへ保存しています...", 1)
    try:
        shutil.copytree(source_path, dest_path)
        log(f"保存完了: {drive_folder}/{dest_dir_name}", 1)

        # Cleanup
        shutil.rmtree(clone_base)

        return {
            'repo': repo,
            'status': 'OK',
            'path': dest_path,
            'files': f_count,
            'size': size_mb
        }
    except Exception as e:
        log(f"【失敗】保存中にエラーが発生しました -> {e}", 1)
        return None

# ------------------------------------------------------------------------------
# 4. 実行ループ
# ------------------------------------------------------------------------------

results = []
for i, target in enumerate(targets, 1):
    res = process_repository(target, i, len(targets))
    if res:
        results.append(res)
    else:
        results.append({'repo': target['repo'], 'status': 'ERROR', 'path': '-', 'files': 0, 'size': 0})
    print()

# ------------------------------------------------------------------------------
# 5. 結果サマリー
# ------------------------------------------------------------------------------

print_separator("=")
log("全処理が完了しました")
print_separator("=")

# 見出しは見やすく英語表記のままにします（日本語だと列がズレやすいため）
print(f"{'STATUS':<8} | {'REPOSITORY':<30} | {'FILES':<8} | {'SIZE (MB)':<10}")
print_separator("-")

total_files = 0
total_size = 0
success_count = 0

for r in results:
    status = r['status']
    repo_short = r['repo'][-30:] if len(r['repo']) > 30 else r['repo']
    files = r['files']
    size = r['size']

    if status == 'OK':
        success_count += 1
        total_files += files
        total_size += size

    print(f"{status:<8} | {repo_short:<30} | {files:<8} | {size:<10.2f}")

print_separator("-")
log(f"合計: {success_count}/{len(targets)} 件成功 ({total_files} ファイル, {total_size:.2f} MB)")
print_separator("=")
print(f"保存場所: {os.path.join(drive_root, drive_folder)}")
print("\n")
